In [1]:
import pandas as pd
import numpy as np

from src.imc24 import score

# Multiply translation vector by a constant

Doesn't change the score

In [2]:
df_sub = pd.read_csv('data/train/train_labels.csv') \
            .query('(dataset != "pond") & (dataset != "lizard")') \
            .sample(n=200)
df_sub = df_sub.rename(columns={'image_name': 'image_path'})


df_preds = df_sub.copy()
translation_cols = ['tx', 'ty', 'tz']
df_preds[translation_cols] = df_preds['translation_vector'].str.split(';', expand=True).astype(float)

rotations_cols = ['r' + ''.join([str(i),str(j)]) for i in range(3) for j in range(3)]
df_preds[rotations_cols] = df_preds['rotation_matrix'].str.split(';', expand=True).astype(float)

df_preds[translation_cols] *= 2

df_preds['rotation_matrix'] = df_preds[rotations_cols].apply(lambda row: 
                 np.array2string(row.values, separator=';') \
                     .replace(' ', '') \
                     .replace('\n', '')[1:-1], 
                 axis=1)
df_preds['translation_vector'] = df_preds[translation_cols].apply(lambda row: 
                 np.array2string(row.values, separator=';') \
                     .replace(' ', '') \
                     .replace('\n', '')[1:-1], 
                 axis=1)

score(df_sub, df_preds)


*** transp_obj_glass_cup ***

mAA: 1.0

*** dioscuri ***

mAA: 1.0

*** church ***

mAA: 1.0

*** multi-temporal-temple-baalshamin ***

mAA: 0.9783

*** transp_obj_glass_cylinder ***

mAA: 1.0


0.9956521739130434

# Add noise to ground truth

Score decreases rapidly

### Rotation with additive noise 1%

In [3]:
df_sub = pd.read_csv('data/train/train_labels.csv') \
            .query('(dataset != "pond") & (dataset != "lizard")') \
            .sample(n=200)
df_sub = df_sub.rename(columns={'image_name': 'image_path'})

df_preds = df_sub.copy()
translation_cols = ['tx', 'ty', 'tz']
df_preds[translation_cols] = df_preds['translation_vector'].str.split(';', expand=True).astype(float)

rotations_cols = ['r' + ''.join([str(i),str(j)]) for i in range(3) for j in range(3)]
df_preds[rotations_cols] = df_preds['rotation_matrix'].str.split(';', expand=True).astype(float)

noise_level = 0.02
df_preds[rotations_cols] += noise_level*2*(np.random.rand(*df_preds[rotations_cols].shape) - 0.5)

df_preds['rotation_matrix'] = df_preds[rotations_cols].apply(lambda row: 
                 np.array2string(row.values, separator=';') \
                     .replace(' ', '') \
                     .replace('\n', '')[1:-1], 
                 axis=1)
df_preds['translation_vector'] = df_preds[translation_cols].apply(lambda row: 
                 np.array2string(row.values, separator=';') \
                     .replace(' ', '') \
                     .replace('\n', '')[1:-1], 
                 axis=1)

score(df_sub, df_preds)


*** transp_obj_glass_cup ***

mAA: 0.697

*** dioscuri ***

mAA: 0.3659

*** church ***

mAA: 0.1583

*** multi-temporal-temple-baalshamin ***

mAA: 0.2982

*** transp_obj_glass_cylinder ***

mAA: 0.7105


0.4460033978226198

### translation with additive noise 1%

In [4]:
df_sub = pd.read_csv('data/train/train_labels.csv') \
            .query('(dataset != "pond") & (dataset != "lizard")') \
            .sample(n=200)
df_sub = df_sub.rename(columns={'image_name': 'image_path'})

df_preds = df_sub.copy()
translation_cols = ['tx', 'ty', 'tz']
df_preds[translation_cols] = df_preds['translation_vector'].str.split(';', expand=True).astype(float)

rotations_cols = ['r' + ''.join([str(i),str(j)]) for i in range(3) for j in range(3)]
df_preds[rotations_cols] = df_preds['rotation_matrix'].str.split(';', expand=True).astype(float)

# normalize translation vectors
df_preds['translation_norm'] = np.sqrt((df_preds[translation_cols]**2).sum(axis=1))
df_preds['translation_norm_max'] = df_preds.groupby('dataset')['translation_norm'].transform('max')
for col in translation_cols:
    df_preds[col] /= df_preds['translation_norm_max']

noise_level = 0.01
df_preds[translation_cols] += noise_level*2*(np.random.rand(*df_preds[translation_cols].shape) - 0.5)

df_preds['rotation_matrix'] = df_preds[rotations_cols].apply(lambda row: 
                 np.array2string(row.values, separator=';') \
                     .replace(' ', '') \
                     .replace('\n', '')[1:-1], 
                 axis=1)
df_preds['translation_vector'] = df_preds[translation_cols].apply(lambda row: 
                 np.array2string(row.values, separator=';') \
                     .replace(' ', '') \
                     .replace('\n', '')[1:-1], 
                 axis=1)

score(df_sub, df_preds)


*** transp_obj_glass_cup ***

mAA: 0.8542

*** church ***

mAA: 0.2114

*** dioscuri ***

mAA: 0.0946

*** multi-temporal-temple-baalshamin ***

mAA: 0.1126

*** transp_obj_glass_cylinder ***

mAA: 0.8417


0.4228966653220384

# Mean of different predictions

The score decreases

In [6]:
datasets = ['dioscuri', 'church', 'multi-temporal-temple-baalshamin']

dfs = []
for dataset in datasets:
    df_sub = pd.read_csv('data/train/train_labels.csv').query(f'dataset == "{dataset}"')
    df_sub = df_sub.rename(columns={'image_name': 'image_path'})
    df_sub['image_path'] = df_sub.apply(lambda row: f'data\\train\\{row["dataset"]}\\images\\' + row['image_path'], axis=1)
    dfs.append(df_sub)
df_sub = pd.concat(dfs)

dfs_1, dfs_2 = [], []
for dataset in datasets:
    dfs_1.append(pd.read_csv(f'data/train/result_{dataset}_pairs0.1x30_disk768_LightGlue_matchesT0.3.csv'))
    dfs_2.append(pd.read_csv(f'data/train/result_{dataset}_pairs0.3x5_aliked1024_LightGlue_matchesT0.01.csv'))

df_1 = pd.concat(dfs_1)
df_2 = pd.concat(dfs_2)

df_preds = df_1.copy()

for df in [df_1, df_2]:
    translation_cols = ['tx', 'ty', 'tz']
    df[translation_cols] = df_preds['translation_vector'].str.split(';', expand=True).astype(float)

    rotations_cols = ['r' + ''.join([str(i),str(j)]) for i in range(3) for j in range(3)]
    df[rotations_cols] = df['rotation_matrix'].str.split(';', expand=True).astype(float)


df_preds[translation_cols] = df_1[translation_cols] + df_2[translation_cols]
df_preds[rotations_cols] = 0.5*(df_1[rotations_cols] + df_2[rotations_cols])


df_preds['rotation_matrix'] = df_preds[rotations_cols].apply(lambda row: 
                 np.array2string(row.values, separator=';') \
                     .replace(' ', '') \
                     .replace('\n', '')[1:-1], 
                 axis=1)
df_preds['translation_vector'] = df_preds[translation_cols].apply(lambda row: 
                 np.array2string(row.values, separator=';') \
                     .replace(' ', '') \
                     .replace('\n', '')[1:-1], 
                 axis=1)

print('-'*100)
print('Prediction 1')
print('Mean score:', score(df_sub, df_1))

print('-'*100)
print('Prediction 2')
print('Mean score:', score(df_sub, df_2))

print('-'*100)
print('Score(prediction1, prediction2)')
print('Mean score:', score(df_1.dropna(), df_2.dropna()))

print('-'*100)
print('Average of predictions')
print('Mean score:', score(df_sub, df_preds))

----------------------------------------------------------------------------------------------------
Prediction 1

*** multi-temporal-temple-baalshamin ***

mAA: 0.241

*** church ***

mAA: 0.2835

*** dioscuri ***

mAA: 0.4055
Mean score: 0.30999579147159
----------------------------------------------------------------------------------------------------
Prediction 2

*** multi-temporal-temple-baalshamin ***

mAA: 0.2205

*** church ***

mAA: 0.285

*** dioscuri ***

mAA: 0.3433
Mean score: 0.28294771052477846
----------------------------------------------------------------------------------------------------
Score(prediction1, prediction2)

*** multi-temporal-temple-baalshamin ***

mAA: 0.7941

*** church ***

mAA: 0.8651

*** dioscuri ***

mAA: 0.7833
Mean score: 0.8141767818238406
----------------------------------------------------------------------------------------------------
Average of predictions

*** multi-temporal-temple-baalshamin ***

mAA: 0.2308

*** church ***

mAA: 0.2